In [175]:
import openai
import pennylane as qml
from PIL import Image
import numpy as np

In [176]:
from key import api_key

In [177]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

def prob_image_arr(arr, openai_api_key):
    # Use OpenAI API to interpret the text
    openai.api_key = openai_api_key

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
            #   {
            #      "role": "system",
            #      "content": "The model is tasked with analyzing a 2D binary array. It should output '1' if the '1's in the array form a perfect rectangle, without any breaks or irregularities. If the '1's do not form a perfect rectangle, the model should output '0'. This binary output should be based solely on the geometric arrangement of '1's within the array."
            #   },
            {
             "role": "system",
               "content": "The model is tasked to receive a 2D binary array consisting of 0s and 1s, with the 1s forming a specific shape. It should analyze this shape and calculate a score from 0 to 1 that indicates how closely this shape resembles a rectangle. A score of 1 denotes an exact rectangle, while a score of 0 indicates no resemblance to a rectangle. The model should focus on the spatial arrangement and continuity of the 1s to determine this score. The output should only be the score with a single number as output without any extra text as explanation."

            # "content": "The model's task is to analyze a 2D binary array and provide a numerical score between 0 and 1. This score represents how closely the '1's in the array resemble a rectangle, with 1 being a perfect rectangle and 0 indicating no resemblance. The model should output only the score, without any accompanying explanation."
            },
            #   {
            #      "role": "system",
            #      "content": "The model is tasked with analyzing a 2D binary array and providing a score between 0 and 1. This score should reflect how closely the arrangement of '1's within the array resembles a rectangle. A score of 1 indicates a perfect rectangle, while a score closer to 0 suggests less resemblance to a rectangle. The model should consider factors such as the continuity, alignment, and proportion of '1's in determining this score."
            #   },
              {"role": "user", "content": f"{arr}"}
            ]
        )
        # print(response)
        content = content = response.choices[0].message.content
        return content
    except Exception as e:
        return f"Error in OpenAI API call: {e}"


In [178]:
# is_arr = prob_image_arr(input_arr, api_key)
# print(is_arr)

In [179]:
import pennylane as qml
from scipy.optimize import minimize
# from pennylane import numpy as np




In [180]:
n_qubits = 2
dev = qml.device('default.qubit', wires=n_qubits)

In [181]:
def find_n_largest_numbers(arr, n):
    # Find the indices of the n largest elements
    indices = np.argsort(arr)[-n:]
    # Find the n largest elements
    largest_elements = arr[indices]
    return largest_elements, indices

def indices_to_binary(indices):
    # Convert each index to binary and ensure it's 8 bits
    return [format(index, f'0{n_qubits}b') for index in indices]

def int_to_binary(n, digits):
    binary_str = format(n, 'b')
    # Pad the binary string with leading zeros to ensure it has 'digits' length
    return binary_str.zfill(digits)

In [182]:
# Define the quantum circuit
@qml.qnode(dev)
def circuit(param,index):
    # print(param)
    index = int_to_binary(index,n_qubits)
    # for i, char in enumerate(index):
    #     if(char=='1'):
    #         qml.PauliX(wires=i)
    #     # qml.Hadamard(wires=i)
            
    qml.StronglyEntanglingLayers(weights=param, wires=range(n_qubits))
    # return qml.probs(wires=range(n_qubits))
    # return qml.expval(qml.PauliZ(0))
    return qml.probs(wires=0)



In [183]:
def convert_to_int(nested_list):
    return [[int(element) for element in sublist] for sublist in nested_list]

def binary_array_to_png_and_display(binary_array, scale_factor=10):
    binary_array = convert_to_int(binary_array)
    # Convert the binary array to a numpy array with values 0 and 255
    image_array = np.array(binary_array) * 255

    # Create an image object from the numpy array
    image = Image.fromarray(np.uint8(image_array), 'L')

    # Scale the image
    original_size = image.size
    scaled_size = tuple([dimension * scale_factor for dimension in original_size])
    scaled_image = image.resize(scaled_size, Image.NEAREST)
    display(scaled_image)


In [184]:
def feature_map(input_arr):
    """Flatten the 2D binary array into a 1D feature map."""
    return [item for sublist in input_arr for item in sublist]

def cosine_similarity(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    vec1, vec2 = np.array(vec1), np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if(norm_vec2*norm_vec1==0): return 0
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

In [185]:
a = int_to_binary(4,5)
for i, char in enumerate(a):
    if(char=='1'):
        print(True)

True


In [186]:
smiley_pattern = np.array([
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]
    ])
smiley_pattern_vec = feature_map(smiley_pattern)

In [187]:
# Define the shape of the parameters
shape_p = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_qubits)

# Define the objective function (to be minimized)
def objective2(params):
    # print(params)
    original_length = np.prod(shape_p)
    # Extract segments for each set of weights
    weights_p1_flat = params[0 : original_length]
    # print('Param')
    # print(weights_p1_flat)
    # weights_p2_flat = params[original_length : 2*original_length]
    # weights_p3_flat = params[2*original_length : 3*original_length]
    # weights_p4_flat = params[3*original_length : 4*original_length]

    # Reshape each segment
    weights_p1 = weights_p1_flat.reshape(shape_p)
    # print(weights_p1)
    # weights_p2 = weights_p2_flat.reshape(shape_p)
    # weights_p3 = weights_p3_flat.reshape(shape_p)
    # weights_p4 = weights_p4_flat.reshape(shape_p)
    # print('===================')
    image_arr = []
    for i in range(n_qubits):
        arr = circuit(weights_p1,i)
        largest_elements, indices = find_n_largest_numbers(arr, 1)
        # Convert indices to binary
        binary_indices = indices_to_binary(indices)
        # print(binary_indices)
        image_arr.append(list(binary_indices[0]))
    print('==============')
    # print(image_arr)
    # Display the image
    binary_array_to_png_and_display(image_arr,scale_factor=20)
    # print(prob_image_arr(image_arr,api_key))
    image_arr = convert_to_int(image_arr)
    generated_img_feature_map = feature_map(convert_to_int(image_arr))
    score = cosine_similarity(smiley_pattern_vec,generated_img_feature_map)
    print(score)
    
    # print("Largest elements:", largest_elements)
    # print("Indices:", indices)
    # print("Indices in binary:", binary_indices)

    return score

def objective(params):
    print(params)
    original_length = np.prod(shape_p)
    # Extract segments for each set of weights
    weights_p1_flat = params[0 : original_length]
    # print('Param')
    # print(weights_p1_flat)
    # weights_p2_flat = params[original_length : 2*original_length]
    # weights_p3_flat = params[2*original_length : 3*original_length]
    # weights_p4_flat = params[3*original_length : 4*original_length]

    # Reshape each segment
    weights_p1 = weights_p1_flat.reshape(shape_p)
  
    return circuit(weights_p1,1)[0]

def callback(params):
    print("Current loss:", objective(params))
    
# Initialize the weights
weights_p1 = np.random.random(size=shape_p)
# weights_p2 = np.random.random(size=shape_p)
# weights_p3 = np.random.random(size=shape_p)
# weights_p4 = np.random.random(size=shape_p)

# Flatten and concatenate the initial parameters
init_params = np.concatenate([weights_p1.flatten()])
print(qml.draw(circuit)(weights_p1,1))

# Optimize using BFGS algorithm
opt = minimize(objective, init_params, method='BFGS',callback=callback)


0: ─╭StronglyEntanglingLayers(M0)─┤  Probs
1: ─╰StronglyEntanglingLayers(M0)─┤       
M0 = 
[[[0.39496535 0.36740515 0.24741111]
  [0.18872373 0.30613348 0.89025315]]]
[0.39496535 0.36740515 0.24741111 0.18872373 0.30613348 0.89025315]
[0.39496537 0.36740515 0.24741111 0.18872373 0.30613348 0.89025315]
[0.39496535 0.36740517 0.24741111 0.18872373 0.30613348 0.89025315]
[0.39496535 0.36740515 0.24741113 0.18872373 0.30613348 0.89025315]
[0.39496535 0.36740515 0.24741111 0.18872375 0.30613348 0.89025315]
[0.39496535 0.36740515 0.24741111 0.18872373 0.30613349 0.89025315]
[0.39496535 0.36740515 0.24741111 0.18872373 0.30613348 0.89025316]
[0.39496536 0.36740515 0.24741112 0.18872375 0.45682056 0.89025316]
[0.39496538 0.36740515 0.24741112 0.18872375 0.45682056 0.89025316]
[0.39496536 0.36740517 0.24741112 0.18872375 0.45682056 0.89025316]
[0.39496536 0.36740515 0.24741114 0.18872375 0.45682056 0.89025316]
[0.39496536 0.36740515 0.24741112 0.18872376 0.45682056 0.89025316]
[0.39496536 0.36

In [188]:
print("Optimized Parameters:", opt.x)
print("Optimized Expectation Value:", opt.fun)

original_length = np.prod(shape_p)
params = opt.x
# Extract segments for each set of weights
weights_p1_flat = params[0 : original_length]
# weights_p2_flat = params[original_length : 2*original_length]
# weights_p3_flat = params[2*original_length : 3*original_length]
# weights_p4_flat = params[3*original_length : 4*original_length]

# Reshape each segment
weights_p1 = weights_p1_flat.reshape(shape_p)
# weights_p2 = weights_p2_flat.reshape(shape_p)
# weights_p3 = weights_p3_flat.reshape(shape_p)
# weights_p4 = weights_p4_flat.reshape(shape_p)
# print(circuit_res(weights_p1,weights_p2,weights_p3,weights_p4)

Optimized Parameters: [0.39496549 0.36740515 0.24741126 0.18872401 3.14159265 0.89025343]
Optimized Expectation Value: 2.553500442031761e-18


In [189]:
res = circuit(weights_p1,0)
# print(res)

In [190]:

# Example array
arr = res
n = 3

# Find n largest numbers and their indices
largest_elements, indices = find_n_largest_numbers(arr, n)

# Convert indices to binary
binary_indices = indices_to_binary(indices)

print("Largest elements:", largest_elements)
print("Indices:", indices)
print("Indices in binary:", binary_indices)


Largest elements: [2.55350044e-18 1.00000000e+00]
Indices: [0 1]
Indices in binary: ['00', '01']
